In [2]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler
import seaborn as sns

import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

In [2]:
def calculate_clustering_metrics(adata, embedding, cluster_key):
    silhouette_avg = silhouette_score(adata.obsm[embedding], adata.obs[cluster_key])
    ch_score = calinski_harabasz_score(adata.obsm[embedding], adata.obs[cluster_key])
    db_score = davies_bouldin_score(adata.obsm[embedding], adata.obs[cluster_key])
    
    return {
        "Silhouette Score": silhouette_avg,
        "Calinski-Harabasz Score": ch_score,
        "Davies-Bouldin Score": db_score
    }

def preprocess(adata, norm_method):
    adata_copy = adata.copy()
    
    # Normalization
    if norm_method == 'total':
        sc.pp.normalize_total(adata_copy, target_sum=1e4)
    elif norm_method == 'size_factors':
        sc.pp.normalize_total(adata_copy, target_sum=1e6)
        sc.pp.log1p(adata_copy)
        adata_copy.obs['size_factors'] = adata_copy.obs['n_counts'] / adata_copy.obs['n_counts'].median()
        adata_copy.X /= adata_copy.obs['size_factors'].values[:, None]
    # elif norm_method == 'pearson_residuals':
    #     # Pearson residuals as an alternative to SCTransform
    #     counts = adata_copy.X.toarray() if sp.sparse.issparse(adata_copy.X) else adata_copy.X
    #     size_factors = counts.sum(axis=1)
    #     size_factors /= np.mean(size_factors)
        
    #     expected = np.outer(size_factors, counts.mean(axis=0))
    #     variance = expected + expected**2 / 10  # assuming dispersion of 0.1
        
    #     pearson_residuals = (counts - expected) / np.sqrt(variance)
    #     adata_copy.layers['pearson_residuals'] = pearson_residuals
    #     adata_copy.X = adata_copy.layers['pearson_residuals']    
    sc.pp.log1p(adata_copy)
    
    # Highly variable genes
    sc.pp.highly_variable_genes(adata_copy, min_mean=0.0125, max_mean=3, min_disp=0.5)
    adata_copy = adata_copy[:, adata_copy.var.highly_variable]
    
    return adata_copy

def batch_correct_and_reduce(adata, batch_key):
    # ComBat batch correction
    sc.pp.combat(adata, key=batch_key)
    
    # PCA
    sc.pp.pca(adata, n_comps=50, use_highly_variable=True, svd_solver='arpack')
    
    # Compute neighbors
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    
    # Compute UMAP for visualization
    sc.tl.umap(adata)
    
    return adata

def cluster_and_evaluate(adata, cluster_method, resolution, batch_key):
    adata_copy = adata.copy()
    
    # Clustering
    cluster_key = f'{cluster_method}_r{resolution}'
    if cluster_method == 'louvain':
        sc.tl.louvain(adata_copy, resolution=resolution, key_added=cluster_key)
    elif cluster_method == 'leiden':
        sc.tl.leiden(adata_copy, resolution=resolution, key_added=cluster_key)
    
    # # Visualization
    # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    # sc.pl.umap(adata_copy, color=batch_key, ax=ax1, show=False, title='Batch')
    # sc.pl.umap(adata_copy, color=cluster_key, ax=ax2, show=False, title=f"{cluster_method}, res={resolution}")
    # plt.tight_layout()
    # plt.show()
    
    # Calculate metrics
    metrics = calculate_clustering_metrics(adata_copy, 'X_umap', cluster_key)
    
    return adata_copy, metrics



In [4]:
# Read data
adata = sc.read('/home/yiyang/github/MD/data/mouse_afterQC.h5ad')

# Define parameters
norm_methods = ['total', 'size_factors']
# norm_methods = ['pearson_residuals']

cluster_methods = ['louvain', 'leiden']
# cluster_methods = ['louvain']

resolutions = [0.5, 1.0]
# resolutions = [0.5]

batch_key = 'sample'  


In [5]:
results = {}
for norm in norm_methods:
    print(f"\nProcessing normalization method: {norm}")
    adata_preprocessed = preprocess(adata, norm)
    
    print(f"Performing batch correction and dimension reduction...")
    adata_corrected = batch_correct_and_reduce(adata_preprocessed, batch_key)
    
    for cluster in cluster_methods:
        for res in resolutions:
            key = f"{norm}_{cluster}_r{res}"
            print(f"Processing {key}...")
            adata_clustered, metrics = cluster_and_evaluate(adata_corrected, cluster, res, batch_key)
            results[key] = metrics
            print(f"Results for {key}:")
            for metric, value in metrics.items():
                print(f"  {metric}: {value}")


Processing normalization method: total
Performing batch correction and dimension reduction...


/home/yiyang/anaconda3/envs/scRNA-seq/lib/python3.11/site-packages/scanpy/preprocessing/_pca.py:385: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/home/yiyang/anaconda3/envs/scRNA-seq/lib/python3.11/site-packages/scanpy/preprocessing/_pca.py:325: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm["X_pca"] = X_pca


Processing total_louvain_r0.5...
Results for total_louvain_r0.5:
  Silhouette Score: 0.3483146131038666
  Calinski-Harabasz Score: 11582.12212423765
  Davies-Bouldin Score: 0.8483002548873645
Processing total_louvain_r1.0...
Results for total_louvain_r1.0:
  Silhouette Score: 0.2834363579750061
  Calinski-Harabasz Score: 11307.433438098104
  Davies-Bouldin Score: 0.9048171244498766
Processing total_leiden_r0.5...


/tmp/ipykernel_1065932/459586877.py:66: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata_copy, resolution=resolution, key_added=cluster_key)


Results for total_leiden_r0.5:
  Silhouette Score: 0.4218815863132477
  Calinski-Harabasz Score: 12951.648958608
  Davies-Bouldin Score: 0.6465722005332804
Processing total_leiden_r1.0...
Results for total_leiden_r1.0:
  Silhouette Score: 0.277815580368042
  Calinski-Harabasz Score: 11476.207362842864
  Davies-Bouldin Score: 1.3744214746514452

Processing normalization method: size_factors
Performing batch correction and dimension reduction...


/home/yiyang/anaconda3/envs/scRNA-seq/lib/python3.11/site-packages/scanpy/preprocessing/_pca.py:385: FutureWarning: Argument `use_highly_variable` is deprecated, consider using the mask argument. Use_highly_variable=True can be called through mask_var="highly_variable". Use_highly_variable=False can be called through mask_var=None
  warn(msg, FutureWarning)
/home/yiyang/anaconda3/envs/scRNA-seq/lib/python3.11/site-packages/scanpy/preprocessing/_pca.py:325: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm["X_pca"] = X_pca


Processing size_factors_louvain_r0.5...
Results for size_factors_louvain_r0.5:
  Silhouette Score: 0.25558939576148987
  Calinski-Harabasz Score: 6293.661010372554
  Davies-Bouldin Score: 1.3193965663859304
Processing size_factors_louvain_r1.0...
Results for size_factors_louvain_r1.0:
  Silhouette Score: 0.23467595875263214
  Calinski-Harabasz Score: 6980.4843153206175
  Davies-Bouldin Score: 1.7944441667470896
Processing size_factors_leiden_r0.5...
Results for size_factors_leiden_r0.5:
  Silhouette Score: 0.2157232165336609
  Calinski-Harabasz Score: 7105.269476970059
  Davies-Bouldin Score: 1.0119014692851303
Processing size_factors_leiden_r1.0...
Results for size_factors_leiden_r1.0:
  Silhouette Score: 0.26930302381515503
  Calinski-Harabasz Score: 7519.138053925071
  Davies-Bouldin Score: 0.9985774990847915


In [6]:
results

{'total_louvain_r0.5': {'Silhouette Score': 0.3483146,
  'Calinski-Harabasz Score': 11582.12212423765,
  'Davies-Bouldin Score': 0.8483002548873645},
 'total_louvain_r1.0': {'Silhouette Score': 0.28343636,
  'Calinski-Harabasz Score': 11307.433438098104,
  'Davies-Bouldin Score': 0.9048171244498766},
 'total_leiden_r0.5': {'Silhouette Score': 0.4218816,
  'Calinski-Harabasz Score': 12951.648958608,
  'Davies-Bouldin Score': 0.6465722005332804},
 'total_leiden_r1.0': {'Silhouette Score': 0.27781558,
  'Calinski-Harabasz Score': 11476.207362842864,
  'Davies-Bouldin Score': 1.3744214746514452},
 'size_factors_louvain_r0.5': {'Silhouette Score': 0.2555894,
  'Calinski-Harabasz Score': 6293.661010372554,
  'Davies-Bouldin Score': 1.3193965663859304},
 'size_factors_louvain_r1.0': {'Silhouette Score': 0.23467596,
  'Calinski-Harabasz Score': 6980.4843153206175,
  'Davies-Bouldin Score': 1.7944441667470896},
 'size_factors_leiden_r0.5': {'Silhouette Score': 0.21572322,
  'Calinski-Harabasz S